In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import pairwise_distances

In [25]:
# fastest source of contractions I could find: https://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python
contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

In [26]:
data = pd.read_csv("raw transcripts/this_american_life.csv")
titles = pd.read_csv("raw data/episode_info_clean.csv")

data.sort_values("episode")
titles.sort_values("episode_number", inplace=True)

df = data.merge(titles, left_on="episode", right_on="episode_number").drop(columns=['episode', "episode_number", "radio_date"])

df.set_index("title", inplace = True)

df.text = df.text.str.lower()
for p in ".,?$%&^*!":
    df.text = df.text.str.replace(p, "")

In [27]:
for k, v in contractions.items():
    df.text = df.text.str.replace(k.lower(), v.lower())

In [29]:
df.to_csv("Cleaned data/TAL_clean.csv")

In [18]:
tvec = TfidfVectorizer(stop_words="english", max_features=600)
port = PorterStemmer()
stops = stopwords.words("english")

In [19]:
for i, episode in enumerate(df.text):
    df.text[i] = " ".join([port.stem(str(s)) for s in str(df.text[i]).split(" ") if not str(s) in stops])

In [20]:
df.head()

,text
title,
New Beginnings,joe franklin readi ira glass oh emce show ir...
Small Scale Sin,ok three boy age 13 15 16 three chose appear ...
Poultry Slam 1995,daniel hous ever sinc girl dinner come someti...
Vacations,thing hawaii go anybodi say paradis liter wor...
Anger and Forgiveness,hi ira glass record 2006 show listen fifth ep...


In [21]:
recommender = pairwise_distances(tvec.fit_transform(df.text), metric="cosine")

In [22]:
rec_df = pd.DataFrame(recommender, index=df.index, columns=df.index)

In [23]:
def recommend(episode, top=11):
    find = rec_df.loc[rec_df.index.str.contains(episode)].index[0]
    return pd.DataFrame(rec_df[find].sort_values()[1:top])

In [32]:
recommend("Bad")

,How Bad Is Bad?
title,
Small Scale Sin,0.077747
Suckers,0.297652
Call For Help,0.302179
Hoaxing Yourself,0.306875
Confessions,0.307118
I Thought I Knew You,0.311834
This I Used to Believe,0.312196
How to Take Money from Strangers,0.315627
Nice Work If You Can Get It,0.317445


## Reading in the corpus for the final recommender test

In [2]:
corpus = pd.read_csv("Recommender files/corpus.csv")

In [3]:
corpus.drop_duplicates(inplace=True)

In [4]:
corpus.reset_index(drop=True, inplace=True)

In [5]:
corpus.set_index("title", inplace=True)

In [6]:
genre_cols = [g for g in list(corpus.columns) if "_genre" in g]

In [7]:
genres = corpus[genre_cols]

In [8]:
import numpy as np
from scipy import sparse

In [9]:
genres_sparse = sparse.csc_matrix(np.array(genres.replace(0, np.NaN)))

In [10]:
tvec = TfidfVectorizer(stop_words="english", max_features=2000, ngram_range=(1,2))

In [11]:
tvec.fit(corpus["text"])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=2000,
                min_df=1, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words='english', strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [12]:
text = tvec.transform(corpus["text"])

In [13]:
rec_data = pairwise_distances(text,metric="cosine")

MemoryError: Unable to allocate array with shape (80099459327,) and data type int64

# Data is is too large for personal computer, final recommender is to be worked on a cloud computing system

In [ ]:
rec_df = pd.DataFrame(rec_data, index=corpus["title"], columns=["title"])

In [ ]:
def recommend(episode, top=11):
    find = rec_df.loc[rec_df.index.str.contains(episode)].index[0]
    return pd.DataFrame(rec_df[find].sort_values()[1:top])

In [ ]:
recommend("Small Scale Sin")